# Ag Census API

In [1]:
import pandas as pd

%load_ext lab_black

In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [3]:
# Docs: https://quickstats.nass.usda.gov/api

In [4]:
url = "https://quickstats.nass.usda.gov/api/api_GET/?key=AC674188-C7A8-3905-9AB7-1FA1DAEDEEF0&commodity_desc=LETTUCE&agg_level_desc=COUNTY&year=2017&format=JSON"

In [5]:
src = pd.read_json(url)

In [6]:
data = []

for r in src["data"]:
    data.append(r)

In [7]:
src = pd.DataFrame(data)

In [8]:
src.short_desc.value_counts()

LETTUCE - ACRES HARVESTED                                          1673
LETTUCE, FRESH MARKET - ACRES HARVESTED                            1634
LETTUCE, FRESH MARKET - OPERATIONS WITH AREA HARVESTED             1634
LETTUCE - OPERATIONS WITH AREA HARVESTED                           1634
LETTUCE, LEAF, FRESH MARKET - OPERATIONS WITH AREA HARVESTED       1465
LETTUCE, LEAF - OPERATIONS WITH AREA HARVESTED                     1465
LETTUCE, LEAF, FRESH MARKET - ACRES HARVESTED                      1465
LETTUCE, LEAF - ACRES HARVESTED                                    1465
LETTUCE, HEAD - ACRES HARVESTED                                     970
LETTUCE, HEAD, FRESH MARKET - OPERATIONS WITH AREA HARVESTED        970
LETTUCE, HEAD, FRESH MARKET - ACRES HARVESTED                       970
LETTUCE, HEAD - OPERATIONS WITH AREA HARVESTED                      970
LETTUCE, ROMAINE, FRESH MARKET - ACRES HARVESTED                    957
LETTUCE, ROMAINE, FRESH MARKET - OPERATIONS WITH AREA HARVESTED 

In [9]:
src.columns = src.columns.str.lower().str.replace(" ", "_", regex=True)

In [10]:
src.rename(
    columns={"state_ansi": "state_fips", "county_ansi": "county_fips"}, inplace=True
)

In [11]:
src_slim = src[
    [
        "year",
        "agg_level_desc",
        "state_name",
        "state_fips",
        "county_name",
        "county_fips",
        "short_desc",
        "domain_desc",
        "value",
    ]
].copy()

In [12]:
src_slim["agg_level_desc"] = src_slim["agg_level_desc"].str.title()
src_slim["county_name"] = src_slim["county_name"].str.title()
src_slim["state_name"] = src_slim["state_name"].str.title()
src_slim["short_desc"] = src_slim["short_desc"].str.title()
src_slim["domain_desc"] = src_slim["domain_desc"].str.title()

In [13]:
src_slim["value"] = (
    src_slim["value"]
    .str.strip(" ")
    .str.replace("(D)", "0", regex=False)
    .str.replace("(Z)", "0", regex=False)
    .str.replace(",", "", regex=False)
    .astype(int)
)

### What types of reports are available? 

In [14]:
src_slim.short_desc.value_counts()

Lettuce - Acres Harvested                                          1673
Lettuce, Fresh Market - Operations With Area Harvested             1634
Lettuce, Fresh Market - Acres Harvested                            1634
Lettuce - Operations With Area Harvested                           1634
Lettuce, Leaf, Fresh Market - Operations With Area Harvested       1465
Lettuce, Leaf - Operations With Area Harvested                     1465
Lettuce, Leaf, Fresh Market - Acres Harvested                      1465
Lettuce, Leaf - Acres Harvested                                    1465
Lettuce, Head - Acres Harvested                                     970
Lettuce, Head, Fresh Market - Operations With Area Harvested        970
Lettuce, Head, Fresh Market - Acres Harvested                       970
Lettuce, Head - Operations With Area Harvested                      970
Lettuce, Romaine - Operations With Area Harvested                   957
Lettuce, Romaine, Fresh Market - Operations With Area Harvested 

In [15]:
types = [
    "Lettuce, Fresh Market - Acres Harvested",
    "Lettuce, Leaf, Fresh Market - Acres Harvested",
    "Lettuce, Head, Fresh Market - Acres Harvested",
    "Lettuce, Romaine, Fresh Market - Operations With Area Harvested",
]
types

['Lettuce, Fresh Market - Acres Harvested',
 'Lettuce, Leaf, Fresh Market - Acres Harvested',
 'Lettuce, Head, Fresh Market - Acres Harvested',
 'Lettuce, Romaine, Fresh Market - Operations With Area Harvested']

---

### Lettuce, Fresh Market: Acres harvested, by county

In [16]:
acres_df = (
    src_slim[
        (src_slim.short_desc == "Lettuce, Fresh Market - Acres Harvested")
        & (src_slim.domain_desc == "Total")
    ]
    .copy()
    .reset_index(drop=True)
)

In [17]:
acres_df.domain_desc.value_counts()

Total    1634
Name: domain_desc, dtype: int64

In [18]:
acres_df.sort_values("value", ascending=False).head(10)

,year,agg_level_desc,state_name,state_fips,county_name,county_fips,short_desc,domain_desc,value
79,2017,County,California,06,Monterey,053,"Lettuce, Fresh Market - Acres Harvested",Total,169496
111,2017,County,California,06,Imperial,025,"Lettuce, Fresh Market - Acres Harvested",Total,35997
117,2017,County,California,06,Santa Barbara,083,"Lettuce, Fresh Market - Acres Harvested",Total,15745
94,2017,County,California,06,Fresno,019,"Lettuce, Fresh Market - Acres Harvested",Total,4702
85,2017,County,California,06,Santa Cruz,087,"Lettuce, Fresh Market - Acres Harvested",Total,4450
114,2017,County,California,06,Riverside,065,"Lettuce, Fresh Market - Acres Harvested",Total,3189
82,2017,County,California,06,San Luis Obispo,079,"Lettuce, Fresh Market - Acres Harvested",Total,2907
81,2017,County,California,06,San Benito,069,"Lettuce, Fresh Market - Acres Harvested",Total,2688
118,2017,County,California,06,Ventura,111,"Lettuce, Fresh Market - Acres Harvested",Total,2257
146,2017,County,Colorado,08,Saguache,109,"Lettuce, Fresh Market - Acres Harvested",Total,1746
